In [9]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import plotly.express as px
from configuration.config import DATA_PATH

In [11]:
df = pd.read_csv(DATA_PATH +'data_new_features.csv')

In [12]:
pd.set_option('display.max_columns', None)
df.head()

,BHCMISID,HealthCenterName,HealthCenterState,FundingCHC,FundingMHC,FundingHO,FundingPH,UrbanRuralFlag,total_hc_funding,total_other_revenue,total_revenue,hispanic_any_race,total_patients,lep_patients,fpl_100_below,fpl_101_to_150,fpl_151_to_200,medicaid_18_up,total_0_17,total_18_up,migrant_patients,homeless_patients,school_based_patients,veteran_patients,public_housing_patients,total_cost_care,total_charges,total_collected,total_sfs,total_clinic_visits,total_virtual_visits,ZipCode_Count,SiteCity_Count,TotalWeeklyHoursOfOperation,hc_grant_revenue_ratio,total_grant_funding,hc_other_grants_ratio,grants_to_rev_ratio,other_to_rev_ratio,other_revenue_category,diverse_patients_sum,diverse_race_proportion,hispanic_proportion,low_income,uninsured,medicaid,public_insurance,private_insurance,poverty_ratio,lep_patients_ratio,fpl_100_below_ratio,low_income_ratio,uninsured_ratio,medicaid_ratio,medicare_18_up_ratio,public_insurance_ratio,private_insurance_ratio,migrant_patients_ratio,homeless_patients_ratio,school_based_patients_ratio,veteran_patients_ratio,public_housing_patients_ratio,total_sites,funding_per_patient,funding_per_patient_group
0,010030,"HOLYOKE HEALTH CENTER, INC.",MA,True,False,False,False,Urban,5721128,14258919.0,28330029,12631.0,19144,8371.0,2706.0,184.0,103.0,8638,5157.0,13987,45.0,2998.0,0.0,332.0,19144.0,NaN,NaN,NaN,NaN,NaN,NaN,50,3.0,175.0,0.201946,11833740.0,0.483459,0.417710,0.503315,1M to 30M,658.0,0.034371,0.659789,2993.0,713.0,13520.0,25.0,1576.0,0.141350,0.437265,0.141350,0.156341,0.037244,0.706226,0.173422,0.001306,0.082323,0.002351,0.156603,0.000000,0.017342,1.000000,4.0,298.847054,50-75%
1,010040,MAINE MOBILE HEALTH PROGRAM INC.,ME,False,True,False,False,Rural,1758567,36578.0,2320228,471.0,850,654.0,768.0,68.0,9.0,36,64.0,786,769.0,8.0,0.0,8.0,0.0,2183264.0,636869.0,2724.0,617158.0,2381.0,192.0,24,2.0,195.0,0.757929,2283650.0,0.770069,0.984235,0.015765,Less than 100K,351.0,0.412941,0.554118,845.0,769.0,60.0,0.0,17.0,0.903529,0.769412,0.903529,0.994118,0.904706,0.070588,0.008235,0.000000,0.020000,0.904706,0.009412,0.000000,0.009412,0.000000,6.0,2068.902353,90-100%
2,010060,"FAIR HAVEN COMMUNITY HEALTH CLINIC, INC.",CT,True,False,False,False,Urban,4569124,270649.0,9064640,19204.0,31918,12686.0,13138.0,3958.0,1685.0,5818,19368.0,12550,3.0,208.0,1937.0,24.0,29342.0,33153465.0,34615790.0,24985087.0,1716660.0,96387.0,27893.0,62,3.0,603.0,0.504060,8575210.0,0.532829,0.946007,0.029858,100K to 1M,7906.0,0.247697,0.601667,18781.0,4467.0,22313.0,0.0,3451.0,0.411617,0.397456,0.411617,0.588414,0.139952,0.699073,0.052823,0.000000,0.108121,0.000094,0.006517,0.060687,0.000752,0.919293,17.0,143.151952,25-50%
3,010070,CORNELL SCOTT HILL HEALTH CORPORATION,CT,True,False,True,True,Urban,6160674,1753132.0,17483690,19860.0,52434,12197.0,22703.0,3842.0,1816.0,26163,8126.0,44308,16.0,2206.0,2125.0,196.0,14603.0,NaN,NaN,NaN,NaN,NaN,NaN,116,5.0,1206.0,0.352367,15458983.0,0.398517,0.884195,0.100272,1M to 30M,18700.0,0.356639,0.378762,28361.0,5438.0,32596.0,0.0,11188.0,0.432982,0.232616,0.432982,0.540889,0.103711,0.621658,0.061220,0.000000,0.213373,0.000305,0.042072,0.040527,0.003738,0.278502,37.0,117.493878,0-25%
4,010120,"CITY OF SPRINGFIELD, MASSACHUSETTS",MA,False,False,True,False,Urban,1842241,0.0,2050671,857.0,2055,461.0,1291.0,63.0,29.0,1321,36.0,2019,85.0,1747.0,0.0,57.0,0.0,2684614.0,1307876.0,726972.0,8876.0,7612.0,1926.0,21,4.0,158.0,0.898360,2042241.0,0.902068,0.995889,0.000000,NaN,483.0,0.235036,0.417032,1383.0,331.0,1357.0,0.0,72.0,0.628224,0.224331,0.628224,0.672993,0.161071,0.660341,0.143552,0.000000,0.035036,0.041363,0.850122,0.000000,0.027737,0.000000,10.0,896.467640,90-100%
